**Dataset**
labeled datasset collected from twitter (Lab 1 - Hate Speech.tsv)

**Objective**
classify tweets containing hate speech from other tweets. <br>
0 -> no hate speech <br>
1 -> contains hate speech <br>


**Evaluation metric**
macro f1 score

### Import used libraries

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
import string
import contractions
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scikeras.wrappers import KerasClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import xgboost as xgb

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)

### Load Dataset

###### Note: search how to load the data from tsv file

In [7]:
df = pd.read_csv("/content/Hate Speech.tsv", sep= "\t", index_col='id')
df.head(10)

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0,bihday your majesty
4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0,factsguide: society now #motivation
6,0,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo
7,0,@user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
8,0,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl
9,0,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦


In [8]:
# a helper functions and imports
from IPython.display import display
def highlight_col(x, df):
    #set by condition
    mask =  df['label'] == 'pos'
    mask2 = df['label'] == 'neg'
    x = pd.DataFrame('', index=df.index, columns=df.columns)
    x.loc[mask] = 'background-color: #e6ffe6'
    x.loc[mask2] = 'background-color: #ffe6e6'
    return x

In [9]:
df.count()

,0
label,31535
tweet,31535


### Data splitting

It is a good practice to split the data before EDA helps maintain the integrity of the machine learning process, prevents data leakage, simulates real-world scenarios more accurately, and ensures reliable model performance evaluation on unseen data.

In [10]:
df.shape

(31535, 2)

In [11]:
train, test = train_test_split(df, test_size=0.2, random_state=1234)


In [12]:
# Split the train into valdtion and train
X_train, X_val, y_train, y_val = train_test_split(train['tweet'], train['label'], test_size=0.2, random_state=42)


In [13]:
# Now, process the test data
X_test = test['tweet']
y_test = test['label']

### EDA on training data

- check NaNs

In [9]:
train.isna().sum()

,0
label,0
tweet,0


- check duplicates

In [10]:
train.duplicated().sum()

1835

- show a representative sample of data texts to find out required preprocessing steps

In [11]:
pd.set_option('display.max_colwidth', 100000)
df_tmp = df.sample(10)
df_tmp.style.apply(lambda x: highlight_col(x, df_tmp), axis=None)

,label,tweet
id,,
5670,1,everyday life for a white south african.
10008,0,@user always seems a shame to build something so nice then box it back down again. good to see you all again
31182,0,@user @user @user you want to see ? this is sad #obama is destroying america from within
23020,0,some people bore me...but im all smiles vibes
25501,0,7 impoant things to allow the #child to be
28379,0,@user having day/night ashes test is disgraceful! sacrilege changing a format that doesn't need changing! what about tradition?
19528,0,@user @user @user @user @user still blaming harper you guys? all of you need professional help. #wow
13805,0,have a great day! #saturday #shopping
26716,0,#justinb gorilla simulator: you need to do to adapt to the environment. the need to tear the city. materia


- check dataset balancing

In [12]:
label= df['label'].value_counts()
label

,count
label,
0,29322
1,2213


There is unbalance in label

- Cleaning and Preprocessing are:
    - 1
    - 2
    - 3
    - ... etc.

### Cleaning and Preprocessing

In [14]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
def preprocess_text(text):
    if pd.isna(text):
        return ""

    # Convert to lowercase
    text = str(text).lower()

    # remove the tab and newline
    text = text.replace('\n', '').replace('\t', ' ')

    # remove extra white space
    text = re.sub(r'\s+', ' ', text)

    # remove contractions
    text = contractions.fix(text)

    # remove the punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # remove emojis
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # remove text elongation
    text = re.sub(r'(.)\1+', r'\1', text)



    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    custom_stopwords = {'also', 'would', 'could', 'may', 'might', 'must', 'need'}
    stop_words.update(custom_stopwords)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

In [15]:
# Compute document vectors by averaging word vectors
def document_vector(doc,model):
    words = [word for word in doc if word in model.wv]
    if words:
        return np.mean(model.wv[words], axis=0)
    else:
        return np.zeros(model.vector_size)



In [16]:

# apply the preprocessing function
X_train_tokens = [preprocess_text(doc) for doc in X_train]
X_val_tokens = [preprocess_text(doc) for doc in X_val]

# Train the Word2Vec model
model = Word2Vec(
    vector_size=100,
    min_count=1,
    workers=4,
    sg=0
)

model.build_vocab(X_train_tokens)
model.train(X_train_tokens, total_examples=len(X_train_tokens), epochs=model.epochs)

# Transform the documents into vectors
X_train_vectors = np.array([document_vector(doc, model) for doc in X_train_tokens])
X_val_vectors = np.array([document_vector(doc, model) for doc in X_val_tokens])


#### Oversampling and Modeling

In [17]:
# Train a RandomForest model
rf_clf = RandomForestClassifier(n_estimators=100, random_state=1234,class_weight='balanced')
rf_clf.fit(X_train_vectors, y_train)

# Evaluate the model
y_pred_val = rf_clf.predict(X_val_vectors)
print("Accuracy on valdtion:", accuracy_score(y_val, y_pred_val))
print("Validation Macro F1 Score on Valdtion:", f1_score(y_val, y_pred_val, average='macro'))

Accuracy on valdtion: 0.9453032104637337
Validation Macro F1 Score on Valdtion: 0.6264099196538937


In [18]:
print("Classification Report on Valdtion:\n", classification_report(y_val, y_pred_val))


Classification Report on Valdtion:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      4723
           1       0.89      0.17      0.28       323

    accuracy                           0.95      5046
   macro avg       0.92      0.58      0.63      5046
weighted avg       0.94      0.95      0.93      5046



#### Evaluate on Test set

In [19]:
# Preprocess the test data
X_test_tokens = [preprocess_text(doc) for doc in X_test]

# Transform the test documents into vectors
X_test_vectors = np.array([document_vector(doc, model) for doc in X_test_tokens])

# Evaluate the model on the test set
y_test_pred = rf_clf.predict(X_test_vectors)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Validation Macro F1 Score on test:", f1_score(y_test, y_test_pred, average='macro'))

Test Accuracy: 0.9416521325511337
Validation Macro F1 Score on test: 0.6108372467984703


In [20]:
print("Test Classification Report on test:\n", classification_report(y_test, y_test_pred))


Test Classification Report on test:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      5880
           1       0.95      0.15      0.25       427

    accuracy                           0.94      6307
   macro avg       0.95      0.57      0.61      6307
weighted avg       0.94      0.94      0.92      6307



#### Extra: use custom scikit-learn Transformers

Using custom transformers in scikit-learn provides flexibility, reusability, and control over the data transformation process, allowing you to seamlessly integrate with scikit-learn's pipelines, enabling you to combine multiple preprocessing steps and modeling into a single workflow. This makes your code more modular, readable, and easier to maintain.

##### link: https://www.andrewvillazon.com/custom-scikit-learn-transformers/

#### Example usage:

**Evaluation metric:**
macro f1 score

Macro F1 score is a useful metric in scenarios where you want to evaluate the overall performance of a multi-class classification model, **particularly when the classes are imbalanced**

![Calculation](https://assets-global.website-files.com/5d7b77b063a9066d83e1209c/639c3d934e82c1195cdf3c60_macro-f1.webp)

In [20]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass  # No need to initialize Word2Vec parameters here

    def preprocess_text(self, text):
        if pd.isna(text):
            return ""

        # Lowercase the text
        text = str(text).lower()

        # Remove tabs, newlines, extra whitespace, contractions, and punctuation
        text = text.replace('\n', '').replace('\t', ' ')
        text = re.sub(r'\s+', ' ', text)
        text = contractions.fix(text)
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Remove URLs, emails, special characters, numbers, emojis, and elongation
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'[^\x00-\x7F]+', '', text)
        text = re.sub(r'(.)\1+', r'\1', text)

        # Tokenization and stopword removal
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        custom_stopwords = {'also', 'would', 'could', 'may', 'might', 'must', 'need'}
        stop_words.update(custom_stopwords)

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
        return tokens

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.preprocess_text(doc) for doc in X]

In [21]:
class Vectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, min_count=1, workers=4, sg=0):
        self.vector_size = vector_size
        self.min_count = min_count
        self.workers = workers
        self.sg = sg
        self.model = None

    def fit(self, X, y=None):
        # Train the Word2Vec model
        self.model = Word2Vec(
            vector_size=self.vector_size,
            min_count=self.min_count,
            workers=self.workers,
            sg=self.sg
        )

        # Build vocab and train model
        self.model.build_vocab(X)
        self.model.train(X, total_examples=len(X), epochs=self.model.epochs)
        return self

    def transform(self, X):
        # Compute document vectors
        def document_vector(doc):
            words = [word for word in doc if word in self.model.wv]
            if words:
                return np.mean(self.model.wv[words], axis=0)
            else:
                return np.zeros(self.vector_size)

        return np.array([document_vector(doc) for doc in X])

In [22]:
class TfidfDocumentVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, max_features=5000, min_df=2, max_df=0.85, ngram_range=(1, 1)):
        self.max_features = max_features
        self.min_df = min_df
        self.max_df = max_df
        self.ngram_range = ngram_range
        self.vectorizer = None

    def fit(self, X, y=None):
        # Convert list of token lists to list of strings for TfidfVectorizer
        docs = [' '.join(doc) for doc in X]

        # Initialize and fit the TF-IDF vectorizer
        self.vectorizer = TfidfVectorizer(
            max_features=self.max_features,
            min_df=self.min_df,
            max_df=self.max_df,
            ngram_range=self.ngram_range
        )
        self.vectorizer.fit(docs)
        return self

    def transform(self, X):
        # Convert list of token lists to list of strings
        docs = [' '.join(doc) for doc in X]

        # Transform documents to TF-IDF vectors
        tfidf_matrix = self.vectorizer.transform(docs)

        # Return as dense array for compatibility with other code
        return tfidf_matrix.toarray()

    def get_feature_names(self):
        """Return feature names (vocabulary terms)"""
        if self.vectorizer is not None:
            return self.vectorizer.get_feature_names_out()
        return None

**You  are doing Great so far!**

### Modelling

#### Extra: use scikit-learn pipline

##### link: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

Using pipelines in scikit-learn promotes better code organization, reproducibility, and efficiency in machine learning workflows.

#### Example usage:

### 1- Logistic regression

In [24]:
pipeline = Pipeline(steps=[
    ('preprocessing', CustomTransformer()),  # Preprocessing text
    ('vectorizing', TfidfDocumentVectorizer()),  # Vectorizing using Word2Vec
    ('model', LogisticRegression(random_state=1234,class_weight = 'balanced')),  # Logistic regression model
])

param_grid = {
    'vectorizing__max_features': [500, 1000, 1500],  # Number of features to keep
    'vectorizing__ngram_range': [(1, 1), (1, 2)],  # Unigrams only or unigrams and bigrams  # Minimum word count for Word2Vec
    'model__C': [0.1, 1, 10],  # Regularization strength for Logistic Regression
    'model__solver': ['liblinear', 'lbfgs'],  # Solvers for Logistic Regression
}

# Define the scoring metric
scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(pipeline, param_grid, scoring=scorer, cv=5, n_jobs=1)


grid_search.fit(X_train, y_train)



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing', CustomTransformer()),
                                       ('vectorizing',
                                        TfidfDocumentVectorizer()),
                                       ('model',
                                        LogisticRegression(class_weight='balanced',
                                                           random_state=1234))]),
             n_jobs=1,
             param_grid={'model__C': [0.1, 1, 10],
                         'model__solver': ['liblinear', 'lbfgs'],
                         'vectorizing__max_features': [500, 1000, 1500],
                         'vectorizing__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, response_method='predict', average=macro))

In [25]:
# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Macro F1 Score:", grid_search.best_score_)

# Use the best model to predict on the test set
y_test_pred = grid_search.predict(X_test)
print("Test Macro F1 Score:", f1_score(y_test, y_test_pred, average='macro'))

Best Parameters: {'model__C': 10, 'model__solver': 'liblinear', 'vectorizing__max_features': 1500, 'vectorizing__ngram_range': (1, 1)}
Best Macro F1 Score: 0.7110790739789471
Test Macro F1 Score: 0.7027808440628176


### 2- RandomFroest Classifer

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessing', CustomTransformer()),  # Preprocessing text
    ('vectorizing', TfidfDocumentVectorizer()),  # Vectorizing using Word2Vec
    ('model', RandomForestClassifier(random_state=1234,n_estimators=200,class_weight = 'balanced')),  # Random Forest model
])

# Define the parameter grid for Random Forest
param_grid = {
    'vectorizing__max_features': [500, 1000, 1500],  # Number of features to keep
    'vectorizing__ngram_range': [(1, 1), (1, 2)],  # Unigrams only or unigrams and bigrams
}

# Define the scoring metric
scorer = make_scorer(f1_score, average='macro')

# Set up the grid search
grid_search = GridSearchCV(pipeline, param_grid, scoring=scorer, cv=5, n_jobs=1)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Macro F1 Score:", grid_search.best_score_)

# Use the best model to predict on the test set
y_test_pred = grid_search.predict(X_test)
print("Test Macro F1 Score:", f1_score(y_test, y_test_pred, average='macro'))

Best Parameters: {'vectorizing__max_features': 1500, 'vectorizing__ngram_range': (1, 1)}
Best Macro F1 Score: 0.7434958035739372
Test Macro F1 Score: 0.7362853593568641


# 3- XGboost

In [24]:
# Define the pipeline with XGBoost
pipeline = Pipeline(steps=[
    ('preprocessing', CustomTransformer()),
    ('vectorizing', TfidfDocumentVectorizer()),
    ('model', xgb.XGBClassifier(random_state=1234,n_estimators=200,class_weight = 'balanced')),
])
# Define the parameter grid for Random Forest
param_grid = {
    'vectorizing__max_features': [500, 1000, 1500],  # Number of features to keep
    'vectorizing__ngram_range': [(1, 1), (1, 2)],  # Unigrams only or unigrams and bigrams
}

# Define the scoring metric
scorer = make_scorer(f1_score, average='macro')

# Set up the grid search
grid_search = GridSearchCV(pipeline, param_grid, scoring=scorer, cv=5, n_jobs=1)

# Fit the grid search
grid_search.fit(X_train, y_train)



# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Macro F1 Score:", grid_search.best_score_)

# Use the best model to predict on the test set
y_test_pred = grid_search.predict(X_test)
print("Test Macro F1 Score:", f1_score(y_test, y_test_pred, average='macro'))




/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:23:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:23:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:23:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:23:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:24:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.



Best Parameters: {'vectorizing__max_features': 1500, 'vectorizing__ngram_range': (1, 1)}
Best Macro F1 Score: 0.7536158181901156
Test Macro F1 Score: 0.7634027024788193


## 3- CNN

In [19]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return [text.lower() for text in X]

# ------------------------------
# 2. Tokenizer & Padder With CNN
# ------------------------------
class TextCNNPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, max_features=10000, maxlen=100):
        self.max_features = max_features
        self.maxlen = maxlen
        self.tokenizer = None

    def fit(self, X, y=None):
        self.tokenizer = Tokenizer(num_words=self.max_features)
        self.tokenizer.fit_on_texts(X)
        return self

    def transform(self, X):
        sequences = self.tokenizer.texts_to_sequences(X)
        return pad_sequences(sequences, maxlen=self.maxlen)

# ------------------------------
# 3. CNN Model Function
# ------------------------------
def build_cnn_model(input_length, num_classes, max_features=10000, embedding_dims=50,
                    filters=64, kernel_size=3, hidden_dims=50, dropout_rate=0.5):
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=embedding_dims, input_length=input_length))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(hidden_dims, activation='relu'))
    model.add(Dropout(dropout_rate))

    if num_classes == 2:
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    else:
        model.add(Dense(num_classes, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

num_classes = len(np.unique(y_train))
maxlen = 100

# Step 1: Apply preprocessing pipeline
preprocess_pipeline = Pipeline([
    ('custom', CustomTransformer()),
    ('token_pad', TextCNNPreprocessor(max_features=10000, maxlen=maxlen))
])

X_train_processed = preprocess_pipeline.fit_transform(X_train, y_train)
X_test_processed = preprocess_pipeline.transform(X_test)

# Step 2: Build & train the model
model = build_cnn_model(input_length=maxlen, num_classes=num_classes)
model.fit(X_train_processed, np.array(y_train), epochs=10, batch_size=32, verbose=1)

# Step 3: Predict & Evaluate
y_pred_probs = model.predict(X_test_processed)
y_pred = (y_pred_probs > 0.5).astype(int).flatten() if num_classes == 2 else np.argmax(y_pred_probs, axis=1)

print("Macro F1 Score:", f1_score(y_test, y_pred, average='macro'))


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


631/631 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9238 - loss: 0.2558
Epoch 2/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9669 - loss: 0.0982
Epoch 3/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9837 - loss: 0.0582
Epoch 4/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9928 - loss: 0.0354
Epoch 5/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9963 - loss: 0.0210
Epoch 6/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9976 - loss: 0.0136
Epoch 7/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9987 - loss: 0.0065
Epoch 8/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9983 - loss: 0.0056
Epoch 9/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9992 - loss: 0.0025
Epoch 10/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9988 - loss: 0.0050
198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Macro F1 Score: 0.8099913432050307


### Conclusion and final results


1- After try the Embedding technics I found that tfidif is the best one of them in this task.


2- After training Logisitic regression, Random Forest, XGboost and CNN model I found That CNN is the best model.

In [25]:
y_pred_probs = model.predict(X_test_processed)
y_pred = (y_pred_probs > 0.5).astype(int).flatten() if num_classes == 2 else np.argmax(y_pred_probs, axis=1)

print("Macro F1 Score for CNN:", f1_score(y_test, y_pred, average='macro'))

198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Macro F1 Score for CNN: 0.8099913432050307


In [26]:
model.save("cnn_text_model.h5")

#### Done!